# Quickstart

The central interface for working with quantum circuits in MQT is the `QuantumComputation` class. It represents quantum circuits as a sequential list of operations. Operations can be directly applied to the `QuantumComputation`: 

In [ ]:
from mqt.core import QuantumComputation

# Build a `QuantumComputation` representing a Bell-state preparation circuit.
nqubits = 2
qc = QuantumComputation(nqubits)

qc.h(0)  # Apply Hadamard gate on qubit 0
qc.cx(0, 1)  # Apply a CNOT (controlled X-Gate) with control on qubit 0 and target on qubit 1

# Get Circuit in OpenQASM 3.0 format
print(qc.qasm3_str())

The circuit class provides a lot of flexibility as every unitary gate can be declared as a controlled gate:

In [ ]:
from mqt.core.operations import Control

nqubits = 2
qc = QuantumComputation(nqubits)

# Controlled Hadamard Gate
qc.ch(0, 1)

# Negatively controlled S-gate: S-Gate on target is performed if control is in |0> state.
qc.cs(Control(0, Control.Type.Neg), 1)

print(qc.qasm3_str())

Providing a set of `Control` objects allows declaring any gate as a multi-controlled gate:

In [ ]:
nqubits = 3
qc = QuantumComputation(nqubits)

# Toffoli gate in mqt-core:
qc.mcx({0, 1}, 2)

# Control type can be individually declared
qc.mcs({Control(0, Control.Type.Neg), Control(1, Control.Type.Pos)}, 2)

print(qc.qasm3_str())

## Layout Information

A `QuantumComputation` also contains information about the mapping of algorithmic (or logical/virtual/circuit) qubits to and from device (or physical) qubits. 
These are contained in the `initial_layout` and `output_permutation` members which are instances of the `Permutation` class. If no layout is given the trivial layout is assumed. 

When printing the OpenQASM representation of the `QuantumComputation` the input and output permutations are given as comments in the first two lines of the QASM string. The format is:

`// i Q_0, Q_1, ..., Q_n` ... algorithmic qubit $i$ is mapped to device qubit $Q_i$.

`// o Q_0, Q_1, ..., Q_n` ... the value of algorithmic qubit $i$ (assumed to be stored in classical bit $c[i]$) is measured at device qubit $Q_i$.


In [ ]:
nqubits = 3
qc = QuantumComputation(nqubits)
qc.initial_layout[2] = 0
qc.initial_layout[0] = 1
qc.initial_layout[1] = 2

qc.output_permutation[2] = 0
qc.output_permutation[0] = 1
qc.output_permutation[1] = 2


print(qc.qasm3_str())

The layout information can also be automatically determined from measurements.

In [ ]:
nqubits = 3
qc = QuantumComputation(nqubits, nqubits)  # 3 qubits, 3 classical bits

qc.h(0)
qc.x(1)
qc.s(2)
qc.measure(1, 0)  # c[0] is measured at qubit 1
qc.measure(2, 1)  # c[1] is measured at qubit 2
qc.measure(0, 2)  # c[2] is measured at qubit 0
qc.initialize_io_mapping()  # determine permutation from measurement

print(qc.qasm3_str())

## Visualizing Circuits

Circuits can be printed in a human-readable format:

In [ ]:
from mqt.core import QuantumComputation

nqubits = 2
qc = QuantumComputation(nqubits, 1)

qc.h(0)
qc.cx(0, 1)
qc.measure(1, 0)

print(qc)

## Operations

The operations in a `QuantumComputation` object are of type `Operation`. Every type of operation in `mqt-core` is derived from this class. Operations can also be explicitly constructed. Each `Operation` has a type in the form of an `OpType`.

### `StandardOperation`
A `StandardOperation` is used to represent basic unitary gates. These can also be declared with arbitrary targets and controls.

In [ ]:
from math import pi

from mqt.core.operations import OpType, StandardOperation

nqubits = 3

# u3 gate on qubit 0 in a 3-qubit circuit
u_gate = StandardOperation(target=0, params=[pi / 4, pi, -pi / 2], op_type=OpType.u)

# controlled x-rotation
crx = StandardOperation(control=Control(0), target=1, params=[pi], op_type=OpType.rx)

# multi-controlled x-gate
mcx = StandardOperation(controls={Control(0), Control(1)}, target=2, op_type=OpType.x)

# add operations to a quantum computation
qc = QuantumComputation(nqubits)
qc.append(u_gate)
qc.append(crx)
qc.append(mcx)

print(qc)

### `NonUnitaryOperation`

A `NonUnitaryOperation` is used to represent operations involving measurements or resets.

In [ ]:
from mqt.core.operations import NonUnitaryOperation

nqubits = 2
qc = QuantumComputation(nqubits, nqubits)
qc.h(0)

# measure qubit 0 on classical bit 0
meas_0 = NonUnitaryOperation(target=0, classic=0)

# reset all qubits
reset = NonUnitaryOperation(targets=[0, 1], op_type=OpType.reset)

qc.append(meas_0)
qc.append(reset)

print(qc.qasm3_str())

### `SymbolicOperation`

A `SymbolicOperation` can represent all gates of a `StandardOperation` but the gate parameters can be symbolic. 
Symbolic expressions are represented in MQT using the `Expression` type, which represent linear combinations of symbolic `Term` objects over some set of `Variable` objects.

In [ ]:
from mqt.core.operations import SymbolicOperation
from mqt.core.symbolic import Expression, Term, Variable

nqubits = 1

x = Variable("x")
y = Variable("y")
sym = Expression([Term(x, 2), Term(y, 3)])
print(sym)

sym += 1
print(sym)

# Create symbolic gate
u1_symb = SymbolicOperation(target=0, params=[sym], op_type=OpType.p)

# Mixed symbolic and instantiated parameters
u2_symb = SymbolicOperation(target=0, params=[sym, 2.0], op_type=OpType.u2)

### `CompoundOperation`

A `CompoundOperation` bundles multiple `Operation` objects together. 

In [ ]:
from mqt.core.operations import CompoundOperation

nqubits = 2
comp_op = CompoundOperation()

# create bell pair circuit
comp_op.append(StandardOperation(0, op_type=OpType.h))
comp_op.append(StandardOperation(target=0, control=Control(1), op_type=OpType.x))

qc = QuantumComputation(nqubits)
qc.append(comp_op)

print(qc)

Circuits can be conveniently turned into operations which allows to create nested circuits:

In [ ]:
from mqt.core import QuantumComputation

nqubits = 2
comp = QuantumComputation(nqubits)
comp.h(0)
comp.cx(0, 1)

qc = QuantumComputation(nqubits)
qc.append(comp.to_operation())

print(qc)

## Interfacing with other SDKs

Since a `QuantumComputation` can be imported from and exported to an OpenQASM 3.0 (or OpenQASM 2.0) string, any library that can work with OpenQASM is easy to use in conjunction with the `QuantumComputation` class. 

In addition, `mqt-core` can import [Qiskit](https://qiskit.org/) [`QuantumCircuit`](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html) objects directly.

In [ ]:
from qiskit import QuantumCircuit

from mqt.core.plugins.qiskit import qiskit_to_mqt

# GHZ circuit in qiskit
qiskit_qc = QuantumCircuit(3)
qiskit_qc.h(0)
qiskit_qc.cx(0, 1)
qiskit_qc.cx(0, 2)

print(qiskit_qc.draw())

mqt_qc = qiskit_to_mqt(qiskit_qc)
print(mqt_qc)